In [ ]:
import os
import gzip
import shutil
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import random
import chardet

In [ ]:
## パラメータ設定
year = 2024   # データの取得年を指定
field_name = "四鳳南喰赤－"     # 取得する対局の種類を指定

In [ ]:
# 2013年以降はgzファイルに圧縮されているので以下の手順で処理

# ① フォルダ内の「scc（鳳凰卓）」から始まる全ファイルを抽出
input_folder = f"raw_data/{year}/"          # https://tenhou.net/sc/raw/ からダウンロードしたログの保存先フォルダ
output_folder = f"extracted_html/{year}/"   # 牌譜URLを示したhtmlの保存先
xml_output_folder = f"xml_files/{year}/"    # 牌譜xmlファイルの保存先

os.makedirs(output_folder, exist_ok=True)
os.makedirs(xml_output_folder, exist_ok=True)

In [ ]:
# ② gzipファイルを解凍してHTMLファイルを保存
def extract_gzip_files(folder):
    html_files = []
    for file_name in os.listdir(folder):
        if file_name.startswith("scc") and file_name.endswith(".html.gz"):
            gzip_path = os.path.join(folder, file_name)
            html_path = os.path.join(output_folder, file_name[:-3])  # .gzを除く
            with gzip.open(gzip_path, "rb") as gz_file:
                with open(html_path, "wb") as html_file:
                    shutil.copyfileobj(gz_file, html_file)
            html_files.append(html_path)
    return html_files

html_files = extract_gzip_files(input_folder)

In [ ]:
# ③ HTMLファイルから条件に合う牌譜URLを取得
def extract_urls_from_html(html_files):
    urls = []
    for html_file in html_files:
        with open(html_file, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")
            
            # <a>タグの前にあるテキストが「四鳳南喰赤－」であればURLを取得
            for element in soup.find_all(string=True):  # テキストノードをチェック
                if field_name in element:
                    # 見つかったテキストノードの直後の<a>タグを取得
                    next_link = element.find_next("a")
                    if next_link and "http://tenhou.net/0/?log=" in next_link.get("href"):
                        urls.append(next_link.get("href"))
                    
    return urls

log_urls = extract_urls_from_html(html_files)

# 結果確認
print(f"抽出されたURL数: {len(log_urls)}")
print(log_urls[:5])  # 最初の5件を表示

In [ ]:
# ④ URLをXMLダウンロード用の形式に変換し、ダウンロード

# XMLダウンロード
def download_xml(log_urls):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # tqdmで進捗バーを表示
    for log_url in tqdm(log_urls, desc="Downloading XMLs", unit="file"):
        # URLをXML用の形式に変換
        xml_url = log_url.replace("http://tenhou.net/0/?log=", "https://tenhou.net/0/log/?")
        log_id = xml_url.split("?")[-1]  # logIDを抽出
        xml_path = os.path.join(xml_output_folder, f"{log_id}.xml")
        
        # 既にダウンロード済みか確認
        if os.path.exists(xml_path):
            continue  # 既にダウンロードされていればスキップ

        try:
            # リクエスト送信
            response = requests.get(xml_url, headers=headers, timeout=10)
            
            if response.status_code == 200 and response.content.strip():  # 正常に内容が取得できた場合
                with open(xml_path, "wb") as file:
                    file.write(response.content)
            else:
                print(f"Failed to download {xml_url} - Status code: {response.status_code}")
        
        except Exception as e:
            print(f"Error downloading {xml_url}: {e}")
        
        # サーバー負荷を避けるための遅延
        # 必須！ サービスに支障をきたさないよう、天鳳のサーバーに過剰な負荷をかけるのはやめましょう
        time.sleep(XXX)

# ダウンロードを実行
download_xml(log_urls)